In [1]:
pip install pandas pyarrow

Note: you may need to restart the kernel to use updated packages.


In [9]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 차원 축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 군집
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth

# 학습 모델 저장을 위한 라이브러리
import pickle

### 회원정보

In [10]:
# 회원정보 7월부터 12월까지 데이터를 불러오기
train_df07 = pd.read_parquet('open/train/1.회원정보/201807_train_회원정보.parquet')
train_df08 = pd.read_parquet('open/train/1.회원정보/201808_train_회원정보.parquet')
train_df09 = pd.read_parquet('open/train/1.회원정보/201809_train_회원정보.parquet')
train_df10 = pd.read_parquet('open/train/1.회원정보/201810_train_회원정보.parquet')
train_df11 = pd.read_parquet('open/train/1.회원정보/201811_train_회원정보.parquet')
train_df12 = pd.read_parquet('open/train/1.회원정보/201812_train_회원정보.parquet')

In [11]:
# 데이터 병합하기
# ignore_index = True : 인덱스를 새로 0부터 재부여한다.
train_df = pd.concat([train_df07, train_df08, train_df09, train_df10, train_df11, train_df12], ignore_index = True)
# 처음과 마지막 5개 행 출력해서 확인하기
# print(train_df.head())
# print(train_df.tail())

In [12]:
# 각 컬럼의 데이터 타입 확인하기
# 문자열 데이터 
# ID, 연령, Segment, 가입통신회사코드, 거주시도명, 직장시도명, _1순위신용체크구분, _2순위신용체크구분, 
# 연회비발생카드수_B0M, 상품관련면제카드수_B0M, 임직원면제카드수_B0M, 우수회원면제카드수_B0M,
# 기타면제카드수_B0M, Life_Stage
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 78 columns):
 #   Column              Dtype  
---  ------              -----  
 0   기준년월                int64  
 1   ID                  object 
 2   남녀구분코드              int64  
 3   연령                  object 
 4   Segment             object 
 5   회원여부_이용가능           int64  
 6   회원여부_이용가능_CA        int64  
 7   회원여부_이용가능_카드론       int64  
 8   소지여부_신용             int64  
 9   소지카드수_유효_신용         int64  
 10  소지카드수_이용가능_신용       int64  
 11  입회일자_신용             int64  
 12  입회경과개월수_신용          int64  
 13  회원여부_연체             int64  
 14  이용거절여부_카드론          int64  
 15  동의여부_한도증액안내         int64  
 16  수신거부여부_TM           int64  
 17  수신거부여부_DM           int64  
 18  수신거부여부_메일           int64  
 19  수신거부여부_SMS          int64  
 20  가입통신회사코드            object 
 21  탈회횟수_누적             int64  
 22  최종탈회후경과월            int64  
 23  탈회횟수_발급6개월이내        int64  
 24  탈회횟수_발급1년이내         int6

In [13]:
# 출력할 최대 행 수 제한 없애기 (None으로 설정)
pd.set_option('display.max_rows', None)

# 결측치 개수 출력 (생략 없이 모두 출력)
print(train_df.isnull().sum())

기준년월                       0
ID                         0
남녀구분코드                     0
연령                         0
Segment                    0
회원여부_이용가능                  0
회원여부_이용가능_CA               0
회원여부_이용가능_카드론              0
소지여부_신용                    0
소지카드수_유효_신용                0
소지카드수_이용가능_신용              0
입회일자_신용                    0
입회경과개월수_신용                 0
회원여부_연체                    0
이용거절여부_카드론                 0
동의여부_한도증액안내                0
수신거부여부_TM                  0
수신거부여부_DM                  0
수신거부여부_메일                  0
수신거부여부_SMS                 0
가입통신회사코드              387570
탈회횟수_누적                    0
최종탈회후경과월                   0
탈회횟수_발급6개월이내               0
탈회횟수_발급1년이내                0
거주시도명                      0
직장시도명                 244969
마케팅동의여부                    0
유효카드수_신용체크                 0
유효카드수_신용                   0
유효카드수_신용_가족                0
유효카드수_체크                   0
유효카드수_체크_가족                0
이용가능카드수_신용체크               0
이용가능카드수_신용    

In [14]:
# 데이터 기준년월, ID, 입회일자_신용은 의미가 없는 데이터라 판단
# 위 컬럼들은 제거한다.
train_df2 = train_df.copy()
train_df3 = train_df2.drop(columns=['기준년월', 'ID', '입회일자_신용'], errors='ignore')

In [18]:
# 문자형 변수 전체에 대해 Label Encoding 수행 (Segment 포함)
for col in train_df3.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    # 결측치가 있으면 문자열로 변환해서 인코딩 가능하게 처리
    train_df3[col] = train_df3[col].astype(str)
    train_df3[col] = le.fit_transform(train_df3[col])

# 숫자형 변수와 인코딩된 문자형 변수 모두 선택
numeric_cols = train_df3.columns.tolist()

# 상관계수 계산
corr_matrix = train_df3[numeric_cols].corr()

# Segment_enc는 기존 'Segment' 컬럼이 인코딩된 것이니, 'Segment'를 인코딩해서 새 컬럼으로 만들었다면 그걸 사용하세요.
# 예를 들어, 만약 'Segment_enc'라는 컬럼이 별도로 있다면 그걸 사용
# 여기서는 'Segment' 컬럼이 인코딩된 상태이므로 'Segment' 컬럼으로 대체

segment_corr = corr_matrix['Segment'].sort_values(ascending=False)

print(segment_corr)

Segment               1.000000
직장시도명                 0.092497
탈회횟수_누적               0.068744
_1순위신용체크구분            0.066816
최종유효년월_신용_이용          0.064195
남녀구분코드                0.060323
마케팅동의여부               0.047710
최종유효년월_신용_이용가능        0.045123
카드신청건수                0.038922
이용거절여부_카드론            0.036400
탈회횟수_발급1년이내           0.032353
연령                    0.031782
거주시도명                 0.028408
탈회횟수_발급6개월이내          0.015520
회원여부_이용가능_카드론         0.013648
최종카드발급일자              0.010829
청구금액_기본연회비_B0M        0.010117
기본연회비_B0M             0.010117
최종탈회후경과월              0.001823
이용가능카드수_체크_가족        -0.001137
유효카드수_체크_가족          -0.004450
연회비발생카드수_B0M         -0.005661
가입통신회사코드             -0.014604
청구금액_제휴연회비_B0M       -0.019886
제휴연회비_B0M            -0.019886
최종카드발급경과월            -0.022261
회원여부_연체              -0.027847
Life_Stage           -0.047118
동의여부_한도증액안내          -0.049472
소지여부_신용              -0.056724
회원여부_이용가능            -0.083298
회원여부_이용가능_CA         -0.087536
수신거부여부_S

In [22]:
corr = train_df3[['이용금액_R3M_신용체크', '이용금액_R3M_신용']].corr()
print(corr)

               이용금액_R3M_신용체크  이용금액_R3M_신용
이용금액_R3M_신용체크       1.000000     0.946124
이용금액_R3M_신용         0.946124     1.000000
